In [ ]:
import airportsdata
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Hourly, Stations
import openmeteo_requests
import requests_cache
from retry_requests import retry
from zoneinfo import ZoneInfo
import os

stations = Stations()
airports = airportsdata.load("IATA")

In [3]:
master_df = pd.read_csv("data/raw-monthly-flight-data/washington/Washington_Flight_Data_July_2023_June_2023.csv")

In [4]:
master_df.drop ('Unnamed: 0', axis = 1, inplace = True)
master_df.columns
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786932 entries, 0 to 786931
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   FL_DATE              786932 non-null  object 
 1   OP_UNIQUE_CARRIER    786932 non-null  object 
 2   TAIL_NUM             786140 non-null  object 
 3   OP_CARRIER_FL_NUM    786932 non-null  int64  
 4   ORIGIN               786932 non-null  object 
 5   ORIGIN_CITY_NAME     786932 non-null  object 
 6   ORIGIN_STATE_NM      786932 non-null  object 
 7   DEST                 786932 non-null  object 
 8   DEST_CITY_NAME       786932 non-null  object 
 9   DEST_STATE_NM        786932 non-null  object 
 10  CRS_DEP_TIME         786932 non-null  int64  
 11  DEP_TIME             774408 non-null  float64
 12  DEP_DELAY            774404 non-null  float64
 13  CRS_ARR_TIME         786932 non-null  int64  
 14  ARR_TIME             773718 non-null  float64
 15  ARR_DELAY        

In [5]:
master_df['FL_DATE_ONLY'] = pd.to_datetime (master_df['FL_DATE']).dt.date

In [6]:
master_df.dropna (subset = ['DEP_TIME'], inplace = True)
master_df.dropna (subset = ['ARR_TIME'], inplace = True)

master_df['DEP_TIME'] = master_df['DEP_TIME'].astype (int)
master_df['ARR_TIME'] = master_df['ARR_TIME'].astype (int)

max_value = master_df['CRS_ARR_TIME'].max ()
master_df.loc[master_df['CRS_ARR_TIME'] == max_value, 'CRS_ARR_TIME'] = 0

max_value = master_df['ARR_TIME'].max ()
master_df.loc[master_df['ARR_TIME'] == max_value, 'ARR_TIME'] = 0

max_value = master_df['DEP_TIME'].max ()
master_df.loc[master_df['DEP_TIME'] == max_value, 'DEP_TIME'] = 0

master_df['CRS_DEP_TIME'] = master_df['CRS_DEP_TIME'].astype (str).str.zfill (4)
master_df['CRS_DEP_TIME'] = pd.to_datetime (master_df['CRS_DEP_TIME'], format = '%H%M').dt.time

master_df['CRS_ARR_TIME'] = master_df['CRS_ARR_TIME'].astype (str).str.zfill (4)
master_df['CRS_ARR_TIME'] = pd.to_datetime (master_df['CRS_ARR_TIME'], format = '%H%M').dt.time

master_df['DEP_TIME'] = master_df['DEP_TIME'].astype (str).str.zfill (4)
master_df['DEP_TIME'] = pd.to_datetime (master_df['DEP_TIME'], format = '%H%M').dt.time

master_df['ARR_TIME'] = master_df['ARR_TIME'].astype (str).str.zfill (4)
master_df['ARR_TIME'] = pd.to_datetime (master_df['ARR_TIME'], format = '%H%M').dt.time

display (master_df[['CRS_DEP_TIME', 'CRS_ARR_TIME', 'DEP_TIME', 'ARR_TIME']])

,CRS_DEP_TIME,CRS_ARR_TIME,DEP_TIME,ARR_TIME
0,23:50:00,05:20:00,00:06:00,05:27:00
1,19:07:00,20:51:00,21:08:00,23:14:00
2,06:00:00,11:33:00,05:55:00,11:30:00
4,19:10:00,21:48:00,19:08:00,21:40:00
6,08:00:00,10:47:00,07:56:00,10:04:00
...,...,...,...,...
786927,10:55:00,12:55:00,11:10:00,13:03:00
786928,13:30:00,16:00:00,13:47:00,16:13:00
786929,16:35:00,18:35:00,17:19:00,19:06:00
786930,19:15:00,22:50:00,19:44:00,23:32:00


In [8]:
master_df['CRS_DEP_TIME_HOUR'] = master_df['CRS_DEP_TIME'].apply (lambda x : round (x.hour + x.minute / 60))
master_df['CRS_ARR_TIME_HOUR'] = master_df['CRS_ARR_TIME'].apply (lambda x : round (x.hour + x.minute / 60))
master_df['DEP_TIME_HOUR'] = master_df['DEP_TIME'].apply (lambda x : round (x.hour + x.minute / 60))
master_df['ARR_TIME_HOUR'] = master_df['ARR_TIME'].apply (lambda x : round (x.hour + x.minute / 60))

In [9]:
master_df = master_df.reset_index()
master_df.head()

,index,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,DEST,DEST_CITY_NAME,...,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FL_DATE_ONLY,CRS_DEP_TIME_HOUR,CRS_ARR_TIME_HOUR,DEP_TIME_HOUR,ARR_TIME_HOUR
0,0,7/1/2021 12:00:00 AM,AA,N103US,2741,GEG,"Spokane, WA",Washington,DFW,"Dallas/Fort Worth, TX",...,NaN,NaN,NaN,NaN,NaN,2021-07-01,24,5,0,5
1,1,7/1/2021 12:00:00 AM,AA,N103US,2829,DFW,"Dallas/Fort Worth, TX",Texas,GEG,"Spokane, WA",...,36.0,0.0,22.0,0.0,85.0,2021-07-01,19,21,21,23
2,2,7/1/2021 12:00:00 AM,AA,N110UW,2369,GEG,"Spokane, WA",Washington,DFW,"Dallas/Fort Worth, TX",...,NaN,NaN,NaN,NaN,NaN,2021-07-01,6,12,6,12
3,4,7/1/2021 12:00:00 AM,AA,N123UW,699,PHX,"Phoenix, AZ",Arizona,GEG,"Spokane, WA",...,NaN,NaN,NaN,NaN,NaN,2021-07-01,19,22,19,22
4,6,7/1/2021 12:00:00 AM,AA,N146AA,1987,ORD,"Chicago, IL",Illinois,SEA,"Seattle, WA",...,NaN,NaN,NaN,NaN,NaN,2021-07-01,8,11,8,10


In [10]:
air_code = np.unique(master_df[['ORIGIN','DEST']].values)

In [11]:
def chunks(lst, n):
  chunk = list()
  i = 0
  while i<len(lst):
    chunk.append(lst[i:i+n])
    i = i+n
  return chunk

temp = list(air_code)
x = chunks(temp,12)

In [12]:
for i in x:
  print(i)

['ABQ', 'ALW', 'ANC', 'ATL', 'AUS', 'AZA', 'BIL', 'BLI', 'BNA', 'BOI', 'BOS', 'BUR']
['BWI', 'BZN', 'CHS', 'CLE', 'CLT', 'CMH', 'CVG', 'DAL', 'DCA', 'DEN', 'DFW', 'DTW']
['EAT', 'ELP', 'EUG', 'EWR', 'FAI', 'FAT', 'FCA', 'FLL', 'GEG', 'GTF', 'HDN', 'HLN']
['HNL', 'IAD', 'IAH', 'ICT', 'IDA', 'IND', 'JAC', 'JFK', 'JNU', 'KOA', 'KTN', 'LAS']
['LAX', 'LIH', 'MCI', 'MCO', 'MDW', 'MFR', 'MIA', 'MKE', 'MRY', 'MSN', 'MSO', 'MSP']
['MSY', 'OAK', 'OGG', 'OKC', 'OMA', 'ONT', 'ORD', 'PAE', 'PDX', 'PHL', 'PHX', 'PIT']
['PSC', 'PSP', 'PUW', 'RDD', 'RDM', 'RDU', 'RNO', 'RSW', 'SAN', 'SAT', 'SBA', 'SBP']
['SEA', 'SFO', 'SIT', 'SJC', 'SLC', 'SMF', 'SNA', 'STL', 'STS', 'SUN', 'TPA', 'TUS']
['YKM']


In [16]:
import time
weather_df = pd.DataFrame()
not_present = list()
count = 1
for i in x:
    print(i)
    weather_df = pd.DataFrame()
    for code in i:
        try:
            lat = airports[f'{code}']['lat']
            lon = airports[f'{code}']['lon']
        except:
            not_present.append(code)
            print("Not present -- ", not_present)
            continue
        cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
        retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
        openmeteo = openmeteo_requests.Client(session = retry_session)
        url = "https://archive-api.open-meteo.com/v1/archive"
        stations = stations.nearby(lat, lon)
        station = stations.fetch(1)
        st = station.reset_index()['id'][0]
        begin_date = datetime(2018, 1, 1)
        end_date = datetime(2023, 12, 31)
        getting_paras = list()
        getting_paras = {"temperature_2m","snowfall","relative_humidity_2m","precipitation","rain","cloud_cover","shortwave_radiation","wind_speed_100m","wind_direction_100m","wind_gusts_10m","snow_depth"}
        params = {
            "latitude": lat,
            "longitude": lon,
            "start_date": "2018-01-01",
            "end_date": "2023-07-31",
            "hourly": getting_paras
            }
        time.sleep(35)
        # Process hourly data. The order of variables needs to be the same as requested.
        responses = openmeteo.weather_api(url, params=params)
        # Process first location. Add a for-loop for multiple locations or weather models
        response = responses[0]
        print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
        print(f"Elevation {response.Elevation()} m asl")
        print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
        print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")
        hourly = response.Hourly()
        parameters_dataset = list()
        for i in range(11):
            parameters_dataset.append(hourly.Variables(i).ValuesAsNumpy())
        hourly_data = {"date": pd.date_range(
            start = pd.to_datetime(hourly.Time(), unit = "s"),
            end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
            freq = pd.Timedelta(seconds = hourly.Interval()),
            inclusive = "left"
        )}
        getting_paras = list(getting_paras)
        for i in range(11):
            hourly_data[getting_paras[i]] = list(parameters_dataset[i])
        hourly_dataframe = pd.DataFrame(data = hourly_data)
        print(f"location code {code} length {len(hourly_dataframe)}")
        hourly_dataframe['airport_code'] = code
        weather_df = pd.concat([hourly_dataframe,weather_df],ignore_index=True)
    weather_df.to_csv(f"data/weather-data-chunks/washington/new_weather_washington_{count}.csv")
    count+=1

['LAX', 'LIH', 'MCI', 'MCO', 'MDW', 'MFR', 'MIA', 'MKE', 'MRY', 'MSN', 'MSO', 'MSP']
Coordinates 33.91915512084961°E -118.39152526855469°N
Elevation 44.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
location code LAX length 48912
Coordinates 21.968364715576172°E -159.3518524169922°N
Elevation 32.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
location code LIH length 48912
Coordinates 39.33216094970703°E -94.71725463867188°N
Elevation 303.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
location code MCI length 48912
Coordinates 28.43585205078125°E -81.30682373046875°N
Elevation 34.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
location code MCO length 48912
Coordinates 41.79261779785156°E -87.78262329101562°N
Elevation 184.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
location code MDW length 48912
Coordinates 42.35500717163086°E -122.8592529296875°N
Elevation 398.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s

OpenMeteoRequestsError: {'error': True, 'reason': 'Hourly API request limit exceeded. Please try again in the next hour.'}

In [ ]:
print(not_present)
len(not_present)

Combining all datafiles

In [ ]:
weather_df = pd.DataFrame()
directory = 'data/weather-data-chunks/washington/Washington_Flight_Data_July_2023_June_2023.csv'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        print(f"reading {f}")
        curr = pd.read_csv(f)
        weather_df = pd.concat([curr,weather_df],ignore_index=True)
        print(f"{f} combined with main file")

In [ ]:
weather_df.to_csv("data/weather-data/Washington_Weather_Data_July_2023_June_2023.csv")